# Case Study: Finetuning with Function Calling for Stock Prices

In this case study, we will demonstrate the utility of function calling with fine-tuned models by addressing a real-world problem. Our goal is to build a chatbot that retrieves stock prices from an external API in response to user inquiries. Using the base model, we encountered two main challenges:
1. The model struggles to distinguish real companies from fake ones.
2. The function calling definitions were excessively long, significantly increasing the tokens per prompt.

We will explore how fine-tuning, combined with function calling, can enhance the model's accuracy and performance. The steps include:
- Building a training dataset.
- Comparing the fine-tuned model to the base model.
- Measuring the improvements from fine-tuning.

Once we have created a fine-tuned model that meets our requirements, we will develop a cost comparison to evaluate the benefits.

### 0. Function Calling:

Function calling allows an LLM to create a call to an external API – it doesn’t execute it. To actually execute the request, you’ll need to extract the function name and arguments from the LLM response and proceed to call the function with those arguments. The function's output is in JSON format, which is then passed back to the LLM to generate an appropriate result message for the user.


### 1. Finetuning a GPT-4o:

Azure OpenAI uses LoRA to fine tune models efficiently. **LoRA (Low-Rank Adaptation)** finetuning of a Large Language Model is a technique used to adapt pre-trained language models to specific tasks efficiently and with fewer computational resources.  

 Instead of adjusting all the model parameters, LoRA introduces a small number of additional parameters (low-rank matrices) that modify the model's behavior. These new parameters are trained while keeping the original model's parameters mostly unchanged. This way, the model can learn the new task without the need for extensive computational resources or time.


### 2. How much will this cost?

For detailed pricing information, visit the [Azure OpenAI Service Pricing](https://azure.microsoft.com/en-us/pricing/details/cognitive-services/openai-service/).

To estimate the cost of our fine-tuning job, we can use the following formula:

`(Training cost per 1K input tokens / 1K) * number of tokens in input file * number of epochs trained`

**Epoch:** A complete iteration through a dataset during the training process.

1. **If the number of epochs is too low:** Your model might be underfitted, meaning it could perform poorly because it hasn't learned enough from the training data. It may not have had enough iterations to effectively learn and adjust its parameters (e.g., weights and biases).

2. **If the number of epochs is too high:** There's a risk of overfitting, where the model becomes too specialized in the training data and performs poorly on unseen data (examples that weren’t in your training dataset).

The number of epochs is a parameter of the fine-tuning job. Usually, 3 epochs is a reasonable number.

**Let's explore our dataset and estimate our fine-tuning costs.**

In [2]:
encoding

<Encoding 'o200k_base'>

In [13]:
import json
import tiktoken
import numpy as np
from typing import List, Dict, Any
from pathlib import Path

encoding = tiktoken.encoding_for_model("gpt-4o")


def num_tokens_from_messages(
    messages: List[Dict[str, Any]],
    tokens_per_message: int = 3,
    tokens_per_name: int = 1,
) -> int:
    num_tokens = tokens_per_message * len(messages) + 3
    for message in messages:
        for key, value in message.items():
            num_tokens += len(
                encoding.encode(value if isinstance(value, str) else str(value))
            )
            if key == "name":
                num_tokens += tokens_per_name
    return num_tokens


files = ["../data/stock-train.jsonl"]

for file in files:
    print(f"Processing file: {file}")
    with Path(file).open("r", encoding="utf-8") as f:
        dataset = [json.loads(line) for line in f]

    total_tokens = [num_tokens_from_messages(ex.get("messages", {})) for ex in dataset]
    test = [ex.get("messages", {}) for ex in dataset]
price = 0.08
training_cost_per_token = price / 1000
num_epochs = 3
total_cost = np.sum(total_tokens) * training_cost_per_token * num_epochs
print(f"Total cost: ${total_cost:.2f}")

Processing file: ../data/stock-train.jsonl
Total cost: $1.56


In [14]:
test

[[{'role': 'system',
   'content': 'You are an assistant that provides functions related to stock prices. '},
  {'role': 'user',
   'content': "What's the highest stock price that ING Group reached in the last 5 days?"},
  {'role': 'assistant',
   'function_call': {'name': 'get_last_nday_stock_price',
    'arguments': '{"symbol": "ING", "period": "5d"}'}}],
 [{'role': 'system',
   'content': 'You are an assistant that provides functions related to stock prices.'},
  {'role': 'user',
   'content': "What was the closing price of Boeing's stock yesterday?"},
  {'role': 'assistant',
   'function_call': {'name': 'get_last_nday_stock_price',
    'arguments': '{"symbol": "BA", "period": "1d"}'}}],
 [{'role': 'system',
   'content': 'You are an assistant that provides functions related to stock prices. '},
  {'role': 'user',
   'content': "What was the highest price that Netflix's stock reached within the last 2 years?"},
  {'role': 'assistant',
   'function_call': {'name': 'get_last_nday_stoc

### 3. Uploading the training and validation data to Azure OpenAI

In [5]:
from pathlib import Path
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv(override=True)

azure_endpoint = os.getenv("AOAI_ENDPOINT")
aoai_api_key = os.getenv("AOAI_API_KEY")
api_version = os.getenv("AOAI_API_VERSION")

client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=aoai_api_key,
    api_version=api_version,
)

training_file_path = "../data/stock-train.jsonl"
validation_file_path = "../data/stock-test.jsonl"

# Upload the training and validation dataset files to Azure OpenAI with the SDK.
training_response = client.files.create(
    file=open(training_file_path, "rb"), purpose="fine-tune"
)
training_file_id = training_response.id

validation_response = client.files.create(
    file=open(validation_file_path, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-d8b3b321faf1473fb2ee24fd42cafcb9
Validation file ID: file-eccda1b783c047afb7d444c3fc341e2f


### 4. Submit your fine-tuning training job

In [8]:
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-4o-mini",
    seed=123,
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)
print(response.model_dump_json(indent=2))

Job ID: ftjob-a9d2614286ad4c4d93c4ec4bcdd14f2e
Status: pending
{
  "id": "ftjob-a9d2614286ad4c4d93c4ec4bcdd14f2e",
  "created_at": 1734459973,
  "error": null,
  "fine_tuned_model": null,
  "finished_at": null,
  "hyperparameters": {
    "n_epochs": -1,
    "batch_size": -1,
    "learning_rate_multiplier": 1
  },
  "model": "gpt-4o-mini-2024-07-18",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": null,
  "seed": 123,
  "status": "pending",
  "trained_tokens": null,
  "training_file": "file-d8b3b321faf1473fb2ee24fd42cafcb9",
  "validation_file": "file-eccda1b783c047afb7d444c3fc341e2f",
  "estimated_finish": null,
  "integrations": null
}


### 5. Track training job status

You should expect to spend 60-90 min running this sample

In [9]:
# Track training status
from IPython.display import clear_output
import time

start_time = time.time()

# Get the status of our fine-tuning job.
response = client.fine_tuning.jobs.retrieve(job_id)

status = response.status

# If the job isn't done yet, poll it every 10 seconds.
while status not in ["succeeded", "failed"]:
    time.sleep(10)

    response = client.fine_tuning.jobs.retrieve(job_id)
    print(response.model_dump_json(indent=2))
    print(
        "Elapsed time: {} minutes {} seconds".format(
            int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)
        )
    )
    status = response.status
    print(f"Status: {status}")
    clear_output(wait=True)

print(f"Fine-tuning job {job_id} finished with status: {status}")

# List all fine-tuning jobs for this resource.
print("Checking other fine-tune jobs for this resource.")
response = client.fine_tuning.jobs.list()
print(f"Found {len(response.data)} fine-tune jobs.")

KeyboardInterrupt: 

Retrieve fine tuned model name

In [5]:
response = client.fine_tuning.jobs.retrieve(job_id)
print(response.model_dump_json(indent=2))
fine_tuned_model = response.fine_tuned_model

{
  "id": "ftjob-c04c792f32f14e099e56862699f4baa2",
  "created_at": 1728569830,
  "error": null,
  "fine_tuned_model": "gpt-4o-mini-2024-07-18.ft-c04c792f32f14e099e56862699f4baa2",
  "finished_at": 1728572547,
  "hyperparameters": {
    "n_epochs": 3,
    "batch_size": 1,
    "learning_rate_multiplier": 1
  },
  "model": "gpt-4o-mini-2024-07-18",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": [
    "file-4d4b0b23e24545a49559777583ae57b9"
  ],
  "seed": 123,
  "status": "succeeded",
  "trained_tokens": 19662,
  "training_file": "file-8e3c156d957f4be691b8c8b57fc1495f",
  "validation_file": "file-619b527be838472aa01e5434e8623683",
  "estimated_finish": null,
  "integrations": null
}


### 6. Deploy fine-tuned model

You can deploy your fine-tuned model using any of the other common deployment methods like Azure OpenAI Studio. Alternatively, you can deploy your fine-tuned model using the Rest API which requires separate authorization, a different API path, and a different API version. 

In [15]:
# Deploy fine-tuned model

import json
import requests

token = os.getenv("TEMP_AUTH_TOKEN")
subscription = os.getenv("AZ_SUBSCRIPTION_ID")
resource_group = os.getenv("AZ_RESOURCE_GROUP_NAME")
resource_name = os.getenv("AZ_AI_SERVICE_NAME")
model_deployment_name = "gpt-4o-mini-2024-07-18-ft"  ### any name you want

deploy_params = {"api-version": "2023-05-01"}
deploy_headers = {
    "Authorization": "Bearer {}".format(token),
    "Content-Type": "application/json",
}

deploy_data = {
    "sku": {"name": "standard", "capacity": 50},
    "properties": {
        "model": {
            "format": "OpenAI",
            "name": "gpt-4o-mini-2024-07-18.ft-c04c792f32f14e099e56862699f4baa2",
            "version": "1",
        }
    },
}
deploy_data = json.dumps(deploy_data)

request_url = f"https://management.azure.com/subscriptions/{subscription}/resourceGroups/{resource_group}/providers/Microsoft.CognitiveServices/accounts/{resource_name}/deployments/{model_deployment_name}"

print("Creating a new deployment...")

r = requests.put(
    request_url, params=deploy_params, headers=deploy_headers, data=deploy_data
)

print(r)
print(r.reason)
print(r.json())

Creating a new deployment...
<Response [201]>
Created
{'id': '/subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/demofinetunning-ncus-rg/providers/Microsoft.CognitiveServices/accounts/demofinetunning-ncus-aiservice/deployments/gpt-4o-mini-2024-07-18-ft', 'type': 'Microsoft.CognitiveServices/accounts/deployments', 'name': 'gpt-4o-mini-2024-07-18-ft', 'sku': {'name': 'standard', 'capacity': 50}, 'properties': {'model': {'format': 'OpenAI', 'name': 'gpt-4o-mini-2024-07-18.ft-c04c792f32f14e099e56862699f4baa2', 'version': '1'}, 'versionUpgradeOption': 'NoAutoUpgrade', 'capabilities': {'chatCompletion': 'true', 'jsonObjectResponse': 'true', 'maxContextToken': '128000', 'maxOutputToken': '16384', 'assistants': 'true'}, 'provisioningState': 'Creating'}, 'systemData': {'createdBy': 'pablocastao@microsoft.com', 'createdByType': 'User', 'createdAt': '2024-10-10T16:01:22.7802221Z', 'lastModifiedBy': 'pablocastao@microsoft.com', 'lastModifiedByType': 'User', 'lastModifiedAt': '2024-

In [14]:
# Azure Management API URL
url = f"https://management.azure.com/subscriptions/{subscription}/resourceGroups/{resource_group}/providers/Microsoft.CognitiveServices/accounts?api-version=2021-04-30"

# Replace 'your_access_token' with your actual Azure AD token
headers = {
    "Authorization": "Bearer {}".format(token),
    "Content-Type": "application/json",
}

# Make the GET request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    resources = response.json()
    print(json.dumps(resources, indent=4))
else:
    print(f"Error: {response.status_code}")
    print(response.text)

{
    "value": [
        {
            "id": "/subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/demofinetunning-ncus-rg/providers/Microsoft.CognitiveServices/accounts/demofinetunning-ncus-aiservice",
            "name": "demofinetunning-ncus-aiservice",
            "type": "Microsoft.CognitiveServices/accounts",
            "etag": "\"04009e3f-0000-0400-0000-6707df1a0000\"",
            "location": "northcentralus",
            "sku": {
                "name": "S0"
            },
            "kind": "AIServices",
            "properties": {
                "endpoint": "https://northcentralus.api.cognitive.microsoft.com/",
                "internalId": "ee4b4bda587c4ce8a8b53cc2ce20b7b2",
                "dateCreated": "2024-10-10T14:04:38.5080292Z",
                "callRateLimit": {
                    "rules": [
                        {
                            "key": "documentTranslation.post",
                            "renewalPeriod": 1,
                     